In [8]:
#required imports
from google.cloud import bigquery
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
# setup your client
# your project id

PROJECT_ID = 'big-query-348313'
client = bigquery.Client(project = PROJECT_ID)

In [14]:
def load_data_from_ref_object(client,dataset_name, table_name):
    dataset_ref= client.dataset(dataset_name)
    dataset= client.get_dataset(dataset_ref)
    
    print('List of all Tables in Dataset ----')
    
    tables = list(client.list_tables(dataset))
    for t in tables:
        print(t.table_id)
    
    table_ref = dataset_ref.table(table_name)
    table = client.get_table(table_ref)
    
    print('Schema of the Table ----')
    print(table.schema)
    # creating table to Pandas data frame
    data = client.list_rows(table).to_dataframe()
    return data
    


In [15]:
#give details of the dataset
data = load_data_from_ref_object(client,'Churn','Churn')
data.head(100)

In [18]:
# load data using Sql query
def load_dataset_from_query(client,query_):
    job= client.query(query_)
    return job.to_dataframe()

In [19]:
# project_id.datasetname.table_name

data = load_dataset_from_query(client,'select * from big-query-348313.Churn.Churn')
data.head()

In [35]:
# comparing exited and not exited

data = load_dataset_from_query(client,'select Exited, count(Exited) as Count from big-query-348313.Churn.Churn group by Exited')
data.head()

In [36]:
# create barchart 

plt.figure(figsize =(8,6))
sns.barplot(data['Exited'], data['Count'])

In [37]:
# corelation of the salary between the Exited and not exited client

data = load_dataset_from_query(client,'select Exited, AVG(EstimatedSalary) as Salary from big-query-348313.Churn.Churn group by Exited')
data.head()


In [39]:
# Query which country has the highest exiting count

data= load_dataset_from_query(client,'SELECT Geography,Exited, count(Exited) as count from big-query-348313.Churn.Churn group by 1,2')
data.head()

In [42]:
# find all exited staff based on the country

data = load_dataset_from_query(client,'select Geography, Gender, count(Exited) as ExitedCount from big-query-348313.Churn.Churn Group by 1,2,Exited having Exited=1')
data.head()

In [43]:
# plot all exited staffs based on Gender

sns.barplot(x='Geography', y='ExitedCount', hue='Gender',data=data)

In [45]:
data = load_dataset_from_query(client,'select Geography, Gender, count(Exited) as NonExitedCount from big-query-348313.Churn.Churn Group by 1,2,Exited having Exited=0')
data.head()

In [46]:
#Non exited counts based on Gender
sns.barplot(x='Geography', y='NonExitedCount', hue='Gender',data=data)